In [132]:
import requests
from bs4 import BeautifulSoup
import json 
import pymysql
import time

In [144]:
def dongaUrl(search):
    url_temp = 'https://www.donga.com/news/search?&query={search}&check_news=1&more=1&sorting=1&search_date=5&v1=20190401&v2=20190930&range=1'.format(search=search)
    for pageNo in range(1, 650, 15):
        url = url_temp  + "&p={pageNo}".format(pageNo=pageNo)
        html = requests.get(url)  
        bs = BeautifulSoup(html.content,"html.parser")
        news_urls = []
        for url_no in range(len(bs.select(".searchList .tit"))):
            news_urls.append(bs.select(".searchList .tit")[url_no].select("a")[0].get("href"))
        dongaInfo(news_urls)

In [146]:
def dongaInfo(news_urls):
    for url_no in range(len(news_urls)):
        url2 = news_urls[url_no]
        html2 = requests.get(url2)
        bs2 = BeautifulSoup(html2.content,"html.parser")
#         time.sleep(2)
        
        try:
            # 제목
            title = bs2.select(".title")[0].text
            
            # 부제목
            if bs2.select(".sub_title") == []:
                sub_title=""
            else:
                sub_title = bs2.select(".sub_title")[0].text
                
            # 날짜
            newsDate = bs2.select(".title_foot .date01")[0].text
            newsDate = newsDate.replace("입력 ", "").replace("-", "")[0:8]

            # 본문
            content = []
            for content_no in range(len(bs2.select(".article_txt"))):
                content.append(bs2.select(".article_txt")[content_no].text)
            content_replace = content[0].replace("\t","").replace("\r","")
            a = content_replace.find('window.')
            b=  content_replace.find('\');')
            contents = content_replace[:a] + content_replace[b+7:]
            a = contents.find('창닫기기사를')
            contents = contents[:a]

            a = contents.find('주요기사')
            b = contents.find('관련기사')
            c = contents.find('더보기')
            if a != -1: # 주요기사 있을 경우    # 62글자
                contents = contents[:a] + contents[a+62:]
            if b != -1: # 관련기사 있을 경우
                contents = contents[:b] + contents[c+3:]
            if sub_title != "": # 부제목 있을 경우
                contents = contents[len(sub_title)+1:]
            contents = contents.replace("\n", "")
            
            print(url2)
            print("제목 : ",title)
            print("부제목 : ",sub_title)
            print(newsDate)
            print("본문:", contents)        
            print("============================================================")
        except:
            print("<<<<<에러 url:>>>>>>", url2)
            continue
        
        dbData = [[url2, title, newsDate, sub_title, contents]]
        connectDB(dbData)

In [142]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'final'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into donga_bs(url, title, date, sub_title, contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()

In [143]:
search = '일본 여행'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
dongaUrl(search)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
https://www.donga.com/news/article/all/20190409/94957262/1
https://www.donga.com/news/article/all/20190408/94952186/1
https://www.donga.com/news/article/all/20190408/94951271/1
https://www.donga.com/news/article/all/20190408/94938802/1
https://www.donga.com/news/article/all/20190407/94937657/1
https://www.donga.com/news/article/all/20190407/94932809/1
https://www.donga.com/news/article/all/20190405/94919430/1
https://www.donga.com/news/article/all/20190405/94919378/1
https://www.donga.com/news/article/all/20190405/94919323/1
https://www.donga.com/news/article/all/20190406/94920162/1
https://www.donga.com/news/article/all/20190404/94902072/1
https://www.donga.com/news/article/all/20190404/94896870/1
https://www.donga.com/news/article/all/20190404/94890045/2
https://www.donga.com/news/article/all/20190404/94884955/1
https://www.donga.com/news/article/all/20190404/94884748/1
0
1
2
3
4
5
https://www.donga.com/news/article/all/20190403/94876080/2
https://w